In [1]:
myRange = spark.range(1000).toDF("number")

In [2]:
divisBy2 = myRange.where("number % 2 = 0")

In [3]:
flightData2015 = spark\
  .read\
  .option("inferSchema", "true")\
  .option("header", "true")\
  .csv("/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv")

In [4]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [5]:
sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")

dataFrameWay = flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .count()

sqlWay.explain()
dataFrameWay.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#69], functions=[finalmerge_count(merge count#87L) AS count(1)#75L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#69, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#69], functions=[partial_count(1) AS count#87L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#69] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>
== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#69], functions=[finalmerge_count(merge count#92L) AS count(1)#82L])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#69, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#69], functions=[partial_count(1) AS count#92L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#69] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>

In [6]:
from pyspark.sql.functions import max

flightData2015.select(max("count")).take(1)

Out[ 6 ]: [Row(max(count)=370002)]

In [7]:
maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()


+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [8]:
from pyspark.sql.functions import desc

flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .show()


+-----------------+-----------------+
DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
 United States| 411352|
 Canada| 8399|
 Mexico| 7140|
 United Kingdom| 2025|
 Japan| 1548|
+-----------------+-----------------+

In [9]:
flightData2015\
  .groupBy("DEST_COUNTRY_NAME")\
  .sum("count")\
  .withColumnRenamed("sum(count)", "destination_total")\
  .sort(desc("destination_total"))\
  .limit(5)\
  .explain()

== Physical Plan ==
TakeOrderedAndProject(limit=5, orderBy=[destination_total#158L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#69,destination_total#158L])
+- *(2) HashAggregate(keys=[DEST_COUNTRY_NAME#69], functions=[finalmerge_sum(merge sum#164L) AS sum(cast(count#71 as bigint))#154L])
 +- Exchange hashpartitioning(DEST_COUNTRY_NAME#69, 200)
 +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#69], functions=[partial_sum(cast(count#71 as bigint)) AS sum#164L])
 +- *(1) FileScan csv [DEST_COUNTRY_NAME#69,count#71] Batched: false, Format: CSV, Location: InMemoryFileIndex[dbfs:/databricks-datasets/definitive-guide/data/flight-data/csv/2015-summary.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>